 ## Exercise 47 - Filter

In [17]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

inputPath = "/data/students/bigdata-01QYD/ex_data/Ex47/data"
outputPathV1 = "res_out_Ex47V1/"
outputPathV2 = "res_out_Ex47V2/"

__input__ : 
* A CSV file containing a list of user profiles : _name, age, gender_

__output__ : 
*  Select male users (gender=“male”), increase by one their age, and store in the output folder name and age of these users sorted by decreasing age and ascending name (if the age value is the same)

### DataFrames

In [7]:
# map into (movieid , userid)
df = spark.read.load(inputPath, format="csv", header=True, inferSchema=True)
df.show()

+-----+---+------+
| name|age|gender|
+-----+---+------+
| Paul| 40|  male|
|David| 15|  male|
|Susan| 40|female|
|Karen| 34|female|
| John| 40|  male|
+-----+---+------+



In [27]:
df_filtered = df.filter('gender="male"')
df_filtered_updated = df_filtered.selectExpr("name", "age+1 as newage")
df_filtered_updated_sorted = df_filtered_updated.sort("age", "name", ascending=[False,True])
df_filtered_updated_sorted.show()

+-----+------+
| name|newage|
+-----+------+
| John|    41|
| Paul|    41|
|David|    16|
+-----+------+



In [28]:
df_filtered_updated_sorted.write.csv(outputPathV1, header=True)

### SQL

In [31]:
df_sql = spark.read.load(inputPath, format="csv", header=True, inferSchema=True)
df_sql.createOrReplaceTempView("people")

df_sql_updated = spark.sql("""  SELECT name, age+1 as newage
                                FROM people
                                WHERE gender='male'
                                SORT BY age desc, name
                            """)

In [30]:
df_sql_updated.show()

+-----+------+
| name|newage|
+-----+------+
| John|    41|
| Paul|    41|
|David|    16|
+-----+------+



In [32]:
df_sql_updated.write.csv(outputPathV2, header=False)